In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import keras
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Layer, BatchNormalization, Activation, InputLayer
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.losses import CategoricalCrossentropy
from keras.optimizers import Adam
from keras.metrics import CategoricalAccuracy, TopKCategoricalAccuracy
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import L2, L1

Dataset Loading

In [ ]:
train_dir = "dataset/Emotions Dataset/Emotions Dataset/train/"
test_dir = "dataset/Emotions Dataset/Emotions Dataset/test/"
# check exist
if os.path.exists(train_dir):
    print("Training directory found.")
else:
    print("Training directory not found.")
    raise FileNotFoundError("Training directory not found.")
if os.path.exists(test_dir):
    print("Testing directory found.")
else:
    print("Testing directory not found.")
    raise FileNotFoundError("Testing directory not found.")

In [ ]:
# # check images size
# df_image_sizes = []
# for class_name in os.listdir(train_dir):
#     class_path = os.path.join(train_dir, class_name)
#     if os.path.isdir(class_path):
#         image_count = len(os.listdir(class_path))
#         for i in range(image_count):
#             sample_image_path = os.path.join(class_path, os.listdir(class_path)[i])
#             sample_image = keras.preprocessing.image.load_img(sample_image_path)
#             df_image_sizes.append(
#                 {
#                     "class": class_name,
#                     "width": sample_image.size[0],
#                     "height": sample_image.size[1],
#                 }
#             )
# df_image_sizes = pd.DataFrame(df_image_sizes)
# df_image_sizes.value_counts()

In [ ]:
CLASS_NAMES = ["angry", "happy", "sad"]
CONFIGURATION = {
    "IMAGE_SIZE": 224,
    "RESCALE_FACTOR": 1.0 / 255,
    "BATCH_SIZE": 32,
    "N_FILTERS": 16,
    "KERNEL_SIZE": 3,
    "N_STRIDES": 1,
    "REGULATION_RATE": 0.001,
    "DROPOUT_RATE": 0.05,
    "POOL_SIZE": 2,
    "N_CLASSES": len(CLASS_NAMES),
    "N_DENSE_1": 100,
    "N_DENSE_2": 10,
    "EPOCHS": 15,
    "LEARNING_RATE": 0.001,
    "VALIDATION_SPLIT": 0.2,
    "PATIENCE": 5,
}

In [ ]:
train_dataset = keras.preprocessing.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    label_mode="categorical",
    class_names=CLASS_NAMES,
    color_mode="rgb",
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"]),
    shuffle=True,
    seed=99,
    validation_split=CONFIGURATION["VALIDATION_SPLIT"],
    subset="training",
)

In [ ]:
val_dataset = keras.preprocessing.image_dataset_from_directory(
    test_dir,
    labels="inferred",
    label_mode="categorical",
    class_names=CLASS_NAMES,
    color_mode="rgb",
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"]),
    shuffle=True,
    seed=99,
)

In [ ]:
# for i in train_dataset.take(1):  # type: ignore
#     images, labels = i
#     print(images.shape)
#     print(labels.numpy())

Dataset Visualization

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):  # type: ignore
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f"{CLASS_NAMES[tf.argmax(labels[i])]}(L:{labels[i].numpy()})")
        plt.axis("off")

Dataset Preparation

In [ ]:
training_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)  # type: ignore
validation_dataset = val_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)  # type: ignore

Modeling

In [ ]:
lenet_model = keras.Sequential(
    [
        InputLayer((CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"], 3)),
        keras.layers.Resizing(CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"]),
        keras.layers.Rescaling(CONFIGURATION["RESCALE_FACTOR"]),
        Conv2D(
            filters=CONFIGURATION["N_FILTERS"],
            kernel_size=CONFIGURATION["KERNEL_SIZE"],
            strides=CONFIGURATION["N_STRIDES"],
            padding="valid",
            activation="relu",
            kernel_regularizer=L2(CONFIGURATION["REGULATION_RATE"]),
        ),
        BatchNormalization(),
        MaxPooling2D(
            pool_size=CONFIGURATION["POOL_SIZE"], strides=CONFIGURATION["N_STRIDES"] * 2
        ),
        Conv2D(
            filters=CONFIGURATION["N_FILTERS"] * 2 + 4,
            kernel_size=CONFIGURATION["KERNEL_SIZE"],
            strides=CONFIGURATION["N_STRIDES"],
            padding="valid",
            activation="relu",
            kernel_regularizer=L2(CONFIGURATION["REGULATION_RATE"]),
        ),
        BatchNormalization(),
        MaxPooling2D(
            pool_size=CONFIGURATION["POOL_SIZE"], strides=CONFIGURATION["N_STRIDES"] * 2
        ),
        Flatten(),
        Dense(
            CONFIGURATION["N_DENSE_1"],
            activation="relu",
            kernel_regularizer=L2(CONFIGURATION["REGULATION_RATE"]),
        ),
        Dropout(CONFIGURATION["DROPOUT_RATE"]),
        Dense(
            CONFIGURATION["N_DENSE_2"],
            activation="relu",
            kernel_regularizer=L2(CONFIGURATION["REGULATION_RATE"]),
        ),
        BatchNormalization(),
        Dropout(CONFIGURATION["DROPOUT_RATE"]),
        Dense(
            CONFIGURATION["N_CLASSES"],
            kernel_regularizer=L2(CONFIGURATION["REGULATION_RATE"]),
            activation="softmax",
        ),
    ]
)
lenet_model.summary()

Training

In [ ]:
loss = CategoricalCrossentropy(from_logits=False)

In [ ]:
# low loss test
y_true = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
y_pred = np.array([[0.9, 0.05, 0.05], [0.1, 0.8, 0.1], [0.2, 0.2, 0.6]])
loss_value = loss(y_true, y_pred).numpy()
print(f"Low loss value: {loss_value}")

# high loss test
y_true = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
y_pred = np.array([[0.1, 0.8, 0.1], [0.3, 0.01, 0.69], [0.9, 0.1, 0.0]])
loss_value = loss(y_true, y_pred).numpy()
print(f"High loss value: {loss_value}")

In [ ]:
metrics = [
    CategoricalAccuracy(name="categorical_accuracy"),
    TopKCategoricalAccuracy(k=2, name="top_2_categorical_accuracy"),
]

In [ ]:
optimizer = Adam(learning_rate=CONFIGURATION["LEARNING_RATE"])

In [ ]:
lenet_model.compile(
    optimizer=optimizer,  # type: ignore
    loss=loss,
    metrics=metrics,
)

In [ ]:
history = lenet_model.fit(
    training_dataset,
    epochs=CONFIGURATION["EPOCHS"],
    validation_data=validation_dataset,
    verbose=1,
)

In [ ]:
# plot training & validation accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history["categorical_accuracy"], label="Train Accuracy")
plt.plot(history.history["val_categorical_accuracy"], label="Validation Accuracy")
plt.title("Model Categorical Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(loc="lower right")
# plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(loc="upper right")
plt.show()

Evaluation

In [ ]:
lenet_model.evaluate(validation_dataset)

In [ ]:
# show false predictions
plt.figure(figsize=(10, 10))
i = 0
for images, labels in validation_dataset:  # type: ignore
    predictions = lenet_model.predict(images)
    for j in range(len(images)):
        if tf.argmax(predictions[j]) != tf.argmax(labels[j]):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[j].numpy().astype("uint8"))
            plt.title(
                f"Predicted: {CLASS_NAMES[tf.argmax(predictions[j])]}, Actual: {CLASS_NAMES[tf.argmax(labels[j])]}"
            )
            plt.axis("off")
            i += 1
            if i >= 9:
                break
    if i >= 9:
        break
plt.show()